In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, jaccard_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
import mlsmote
from skmultilearn.problem_transform import BinaryRelevance
import json
import matplotlib.pyplot as plt
from itertools import product
import seaborn as sns
from tqdm import tqdm
from inspect import signature
from tensorflow.estimator import DNNClassifier

In [ ]:
# Load the dataset
data = pd.read_csv('ai4i2020.csv',index_col=None)


In [ ]:
no_data_entries = data.shape[0]
repetition_class_ratio= data[['TWF','HDF','PWF','OSF','RNF']].sum(axis=0)/no_data_entries
print(repetition_class_ratio)

In [ ]:
non_failure_samples_no= no_data_entries-data[["Machine failure"]].sum(axis=0).values[0]
print(non_failure_samples_no)

In [ ]:
classifiers = [
    XGBClassifier,
    LGBMClassifier,
    CatBoostClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier,
    LogisticRegression,
    SVC,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    MLPClassifier
]

hyperparameters = [
    { 
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.1, 0.5, 1.0],
        'n_estimators': [50, 100, 200],
        'gamma': [0.0, 0.1, 0.5],
        'subsample': [0.5, 0.8, 1.0],
        'colsample_bytree': [0.5, 0.8, 1.0],
        'tree_method': ['gpu_hist'],  # Use GPU for XGBoost
        'gpu_id': [0]  # Set GPU ID to 0
    },
    {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.5, 1.0],
        'n_estimators': [50, 100, 200],
        'num_leaves': [31, 62, 127],
        'subsample': [0.5, 0.8, 1.0],
        'colsample_bytree': [0.5, 0.8, 1.0],
        'reg_lambda': [0.0, 0.1, 0.5],
        'device': ['gpu'],  # Use GPU for LightGBM
        'gpu_device_id': [0],  # Set GPU ID to 0
        'verbose': [-1],  # Disable text information
        'silent': [True]  # Disable text information (alternative)
    },
    {
        'depth': [3, 5, 7],
        'learning_rate': [0.1, 0.5, 1.0],
        'n_estimators': [50, 100, 200],
        'l2_leaf_reg': [0.0, 0.1, 0.5],
        'task_type': ['GPU'],  # Use GPU for CatBoost
        'verbose': [False]  # Disable text information
    },
    {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5, 10]
    },
    {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.1, 0.5, 1.0],
        'max_depth': [3, 5, 7],
        'subsample': [0.5, 0.8, 1.0]
    },
    {
        'C': [0.1, 1.0, 10.0]
    },
    {
        'kernel': ['rbf', 'linear'],
        'C': [0.1, 1.0, 10.0],
        'gamma': ['scale', 'auto']
    },
    {
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5, 10]
    },
    {
        'n_neighbors': [3, 5, 10],
        'weights': ['uniform', 'distance']
    },
    {
        'hidden_layer_sizes': [(50, 50), (100, 100), (200, 200)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'invscaling']
    }
]

In [ ]:

# Encode categorical feature
type_dict = {"L": 1, "M": 2, "H": 3}
data['Type'] = data['Type'].map(type_dict)

# Separate features and labels
X = data.drop(['UDI', 'Product ID', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)
y = data[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

repetition_class_ratio_before_da= y_train.sum(axis=0)/y_train.shape[0]
print("before MLSMOTE \n", repetition_class_ratio_before_da)


# Infer the categorical feature from the data columns
categorical_features = [i for i, col in enumerate(X.columns) if col == 'Type']

# Create an instance of MLSMOTE
mlsmote = mlsmote.MLSMOTE(categorical_features=categorical_features,input_columns=X_train.columns, label_columns=y_train.columns,random_state=77)

# Resample the data using MLSMOTE
X_train_res, y_train_res = mlsmote.fit_resample(X_train, y_train)   

# Convert X_train_res and y_train_res to DataFrames
X_train = pd.DataFrame(X_train_res, columns=X_train.columns)
y_train = pd.DataFrame(y_train_res, columns=y_train.columns)


repetition_class_ratio_after_da= y_train.sum(axis=0)/y_train.shape[0]
print("after MLSMOTE \n", repetition_class_ratio_after_da)




# Standardize the features
scaler = StandardScaler()
# scaler = MinMaxScaler()

# Scale the continuous features
X_train_res_continuous = scaler.fit_transform(X_train.drop('Type', axis=1))
X_test_continuous = scaler.transform(X_test.drop('Type', axis=1))

# Concatenate the categorical feature with the scaled continuous features
X_train = np.concatenate((X_train[['Type']], X_train_res_continuous), axis=1)
X_test = np.concatenate((X_test[['Type']], X_test_continuous), axis=1)



best_score = 0
best_params = None
best_classifier = None

results = []
for classifier, hyperparameter in zip(classifiers, hyperparameters):
    hyperparameter_combinations = [dict(zip(hyperparameter.keys(), v)) for v in product(*hyperparameter.values())]
    for hyperparameter_combination in tqdm(hyperparameter_combinations, desc=f"Training {classifier.__name__}"):
        clf_params = hyperparameter_combination.copy()
        if 'random_state' in signature(classifier).parameters:
            clf_params['random_state'] = 77
        clf = BinaryRelevance(classifier(**clf_params))
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        micro_f1 = f1_score(y_test, y_pred, average='micro')
        jaccard_sim = jaccard_score(y_test, y_pred, average='micro')
        score = micro_f1 * jaccard_sim
        results.append({
            'classifier': classifier.__name__,
            'hyperparameters': hyperparameter_combination,
           'micro_f1': micro_f1,
            'jaccard_sim': jaccard_sim,
           'score': score
        })

with open('results.json', 'w') as f:
    json.dump(results, f, indent=4)



In [ ]:
with open('results.json') as f:
    data = json.load(f)

scores_with_indices = [(i, item['score']) for i, item in enumerate(data)]
top_10_scores_with_indices = sorted(scores_with_indices, key=lambda x: x[1], reverse=True)[:10]

for i, score in top_10_scores_with_indices:
    print(f"Index: {i}, Score: {score}")
best_params=data[top_10_scores_with_indices[0][0]]['hyperparameters']
best_classifier=data[top_10_scores_with_indices[0][0]]['classifier']

In [ ]:
print(f"Best classifier: {best_classifier}", f", Best hyperparameters: {best_params}")

In [ ]:
# Train the best classifier
best_classifier = eval(best_classifier)
clf = BinaryRelevance(best_classifier(**best_params))
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)
# Convert sparse matrix to dense NumPy array
y_pred_dense = y_pred.toarray()

# add non-failure column to the prediction
y_pred_non_failure = ~np.any(y_pred_dense, axis=1)
y_pred_dense = np.hstack((y_pred_dense, y_pred_non_failure.reshape(-1, 1)))
y_pred_dense = np.array(y_pred_dense)

# add non-failure column to the ground truth
y_test_non_failure = ~np.any(y_test, axis=1)
y_test = np.hstack((np.array(y_test), np.array(y_test_non_failure).reshape(-1, 1)))
y_test_cr = np.array(y_test)

# Calculate confusion matrix and classification report
print("Confusion Matrix:")
for i, label_name in enumerate(['TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Non-failure']):
    print(f"{label_name}:")
    print(confusion_matrix(y_test_cr[:, i], y_pred_dense[:, i]))
    print()

print("Classification Report:")
print(classification_report(y_test_cr, y_pred_dense, target_names=['TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Non-failure']))

# Classification Report(10th best model):
              precision    recall  f1-score   support

         TWF       0.50      0.11      0.18         9
         HDF       1.00      0.91      0.95        22
         PWF       1.00      0.83      0.91        12
         OSF       0.94      0.89      0.91        18
         RNF       0.00      0.00      0.00         4
         Non-failure       0.99      1.00      1.00      1940
         micro avg       0.99      0.99      0.99      2005
         macro avg       0.74      0.62      0.66      2005
         weighted avg       0.99      0.99      0.99      2005
         samples avg       0.99      0.99      0.99      2005

# Classification Report(Before MLSMOTE):
              precision    recall  f1-score   support

         TWF       0.00      0.00      0.00         9
         HDF       1.00      0.95      0.98        22
         PWF       0.75      1.00      0.86        12
         OSF       1.00      0.94      0.97        18
         RNF       0.00      0.00      0.00         4
         Non-failure       0.99      1.00      1.00      1940
         micro avg       0.99      0.99      0.99      2005
         macro avg       0.62      0.65      0.63      2005
         weighted avg       0.98      0.99      0.99      2005
         samples avg       0.99      0.99      0.99      2005

# Classification Report(After MLSMOTE):
              precision    recall  f1-score   support

         TWF       0.33      0.11      0.17         9
         HDF       1.00      0.95      0.98        22
         PWF       0.80      1.00      0.89        12
         OSF       0.94      0.83      0.88        18
         RNF       0.00      0.00      0.00         4
         Non-failure       0.99      1.00      1.00      1940
         micro avg       0.99      0.99      0.99      2005
         macro avg       0.68      0.65      0.65      2005
         weighted avg       0.99      0.99      0.99      2005
         samples avg       0.99      0.99      0.99      2005

In [ ]:
conf_mat = np.zeros((6, 6))

for i in range(6):
    for j in range(6):
        conf_mat[i, j] = np.sum((y_test[:, i] == 1) & (y_pred_dense[:, j] == 1))

label_names = ['TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Non-failure']

plt.figure(figsize=(10, 10))
sns.heatmap(conf_mat, annot=True,fmt=".0f", cmap='Blues',xticklabels=label_names, yticklabels=label_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion matrix')
plt.show()

In [ ]:
# find acc
acc = np.sum(np.diag(conf_mat)) / np.sum(conf_mat)
print("Accuracy: ", acc)